In [ ]:
import os
import utils
import connectome_create
# viz_method = one of ['itkwidgets', 'vtk']
viz_method = 'vtk'

# import some of our favorite packages
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import re
import warnings

%matplotlib inline
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import AgglomerativeClustering
# import nglui.statebuilder as ngstbld

# # this is the EM specific package for querying the EM data
from caveclient import CAVEclient


# from meshparty import trimesh_io, trimesh_vtk
# from meshparty import skeletonize, skeleton_io, skeleton
# import cloudvolume

%load_ext autoreload
%autoreload 2

In [ ]:
# from pathlib import Path
# import json

# with open(Path.home() / '.cloudvolume/secrets/'/'cave-secret.json') as f:
#         tokens = json.load(f)
        
# seg_source = 'graphene://https://cave.fanc-fly.com/segmentation/table/mar2021_prod'
# cv = cloudvolume.CloudVolume(cloudpath=seg_source, use_https=True, secrets=tokens)

In [ ]:
datastack_name = 'fanc_production_mar2021'

client = CAVEclient(datastack_name)

In [ ]:
# client = CAVEclient()

# # if not os.path.isfile(os.path.expanduser("~/.cloudvolume/secrets/cave-secret.json")):
# client.auth.get_new_token(open=True)

In [ ]:
# if you have not yet setup this computer, uncomment this below line
# paste the token from the website in, and run the line

# client.auth.save_token(token="c14cd7a3e18a1a697716a399afbf5778", overwrite=True)

# then comment or delete the line as you don't need to run it on this computer  again

In [ ]:
pre_to_mn_df = connectome_create.load_pre_to_mn_df(ext='matched_to_pool')
pre_to_mn_df.shape

# Add local neuron classification and hemilineage to matrix

In [ ]:
timestamp = connectome_create.get_timestamp()

# pmn_annotation_table = client.materialize.query_table('t1l_local_premotor_table_v0',timestamp=timestamp)
# pmn_annotation_table = client.materialize.query_table('t1l_local_premotor_table_v1',timestamp=timestamp)
# pmn_annotation_table = client.materialize.query_table('t1l_local_premotor_table_v2',timestamp=timestamp)
pmn_annotation_table = client.materialize.query_table('t1l_local_premotor_table_v3',timestamp=timestamp)

pmn_annotation_table.classification_system.value_counts()
pmn_annotation_table.cell_type.value_counts()

In [ ]:
utils.save_df_csv(pmn_annotation_table,'t1l_local_premotor_table_v2_current')

In [ ]:
# just add classifications and hemilineage to the index, then put the index back on and reorder
pmn_index = pre_to_mn_df.index.to_frame()
pmn_index = pmn_index.set_index(keys=['segID'],drop=False)

pmn_index['classification_system'] = None
pmn_index['cell_type'] = None

for clsys in pmn_annotation_table.classification_system.unique():
    pmn_index.loc[pmn_index.segID.isin(pmn_annotation_table.loc[pmn_annotation_table.classification_system==clsys,'pt_root_id']),'classification_system'] = clsys

for ctype in pmn_annotation_table.cell_type.unique():
    pmn_index.loc[pmn_index.segID.isin(pmn_annotation_table.loc[pmn_annotation_table.cell_type==ctype,'pt_root_id']),'cell_type'] = ctype

pre_to_mn_df.index = pd.MultiIndex.from_frame(pmn_index)
pre_to_mn_df = pre_to_mn_df.reorder_levels(['cell_class','preferred_pool','classification_system','cell_type','segID'],'index')
pre_to_mn_df.head()

# Add neurotransmitters

## Create lookuptable

In [ ]:
cell_types = pre_to_mn_df.index.get_level_values('cell_type').unique().to_numpy()
nt_table = pd.read_csv('./annotations_hl/LacinHLTable.csv')
import re
pattern = re.compile("R[0-9]")

hl_lut = {'cell_type':cell_types,'hemilineage':cell_types}
hl_lut = pd.DataFrame(data=hl_lut)

for idx,r in hl_lut.iterrows():
    try:
        if np.isnan(r.hemilineage):
            continue
    except TypeError:
        pass
    hl = r.hemilineage

    if hl[0:2]=='RD':
        hl = '24B'
        # print('string was RD, now {}'.format(hl))

    if pattern.match(hl) or hl=='RVD' or hl=='Rcore_':
        # print('string was {}, now 8A'.format(hl))
        hl = '8A'

    if not hl.isalnum():
        hl = hl[0:-1]
        # print(hl)
    
    if not hl.isalnum():
        hl=hl[0:hl.find('_')]
        # print(hl)

    if hl == '9Ac':
        hl='9A'
        # print(hl)

    hl_lut.loc[idx,'hemilineage'] = hl

# merge with neurotransmitter table to get a lookuptable
hl_lut = hl_lut.merge(nt_table,how='outer',left_on='hemilineage',right_on='HL')
hl_lut = hl_lut.loc[~hl_lut.hemilineage.isna(),:]

hl_lut_reduced = hl_lut[['cell_type','NT']]

In [ ]:
# just operate on the index, where the nt will be
pmn_index_df = pre_to_mn_df.index.to_frame().reset_index(drop=True)

# join the index on cell_type, with NT
pmn_index_df = pmn_index_df.join(hl_lut_reduced.set_index('cell_type'),how='left',on='cell_type')

# reorder the matrix
pmn_index_df = pmn_index_df[['cell_class','preferred_pool','NT','classification_system','cell_type','segID']]
pmn_index_df.NT.value_counts()

In [ ]:
# check if there are some local neurons that don't have hemilineages
pmn_index_df.loc[(pmn_index_df.cell_class=='local') & (pmn_index_df.classification_system.isna())].segID.values

# 8/3 just updated the premotor neuron table to add the segIDs I found earlier

# Now add the sensory neuron classifications

In [ ]:
all_sensory = connectome_create.get_unduplicated_sensory_axon_table(client)

all_sensory = all_sensory[['pt_root_id','classification_system','cell_type']]
all_sensory = all_sensory.rename({'pt_root_id':'segID'},axis=1)

# join the index on cell_type, with NT
pmn_index_df = pmn_index_df.merge(all_sensory,how='left',on='segID',suffixes=['_x','_y'])

sens_rows = (pmn_index_df.cell_class == 'sensory') & (pmn_index_df.classification_system_x.isna())
pmn_index_df.loc[sens_rows,'classification_system_x'] = pmn_index_df.loc[sens_rows].classification_system_y
pmn_index_df.loc[sens_rows,'cell_type_x'] = pmn_index_df.loc[sens_rows].cell_type_y

pmn_index_df.loc[pmn_index_df.cell_class == 'sensory','NT'] = 'Ach'
pmn_index_df = pmn_index_df.drop(['classification_system_y','cell_type_y'],axis=1)
pmn_index_df = pmn_index_df.rename(columns={'classification_system_x':'classification_system','cell_type_x':'cell_type'})


In [ ]:
pre_to_mn_df.index = pd.MultiIndex.from_frame(pmn_index_df)

# Save the data frame

In [ ]:
connectome_create.save_pre_to_mn_df(pre_to_mn_df,ext='matched_typed_with_nt')

## Now print the segids for 13A as an example, then color them in neuroglancer

In [ ]:
All = slice(None)
pmn_index_df = pmn_index.set_index(keys=['cell_class','preferred_pool','classification_system','cell_type'],drop=False)
eg13A = pmn_index_df.loc[('local',All,'D')]

for pl in eg13A.preferred_pool.unique():
    print(eg13A.loc[(All,pl,All),'segID'])


In [ ]:
pmn_index.classification_system.value_counts()

# Count the number of synapes from different types to MFT, SET, Aux B, Aux E

In [ ]:
# mpool_dict = utils.get_motor_pool_tuple_dict()
# pool_keys = [
#     'thorax_swing',
#     'tibia_extensor',
#     'main_tibia_flexor',
#     'auxiliary_tibia_flexor_B',
#     'auxiliary_tibia_flexor_E',
#     ]

# local = pre_to_mn_df.loc['local']
# for pk in pool_keys:
#     local_to_pool = local.loc[:,mpool_dict[pk]]

#     labels = sorted(local_to_pool.index.unique(level='classification_system'))
#     sizes = []
#     for lbl in labels:
#         ttlsyn = local_to_pool.loc[(All,lbl),:].sum().sum()
#         sizes = sizes+[ttlsyn]


#     fig1, ax1 = plt.subplots()
#     ax1.pie(sizes, explode=None, labels=labels, autopct='%1.0f%%',
#             shadow=False, startangle=90)
#     ax1.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.

#     plt.show()

#     fname = './figpanels/{}_hl_input_pie.svg'.format(pk)
#     fig1.savefig(fname,format='svg')

# Find untyped neurons to type

In [ ]:
pmn_index.loc[(pmn_index.cell_class=='local') & pmn_index.classification_system.isnull()].segID.to_numpy()

In [ ]:
untyped_inter = pmn_index.loc[(pmn_index.cell_class=='intersegmental') & pmn_index.classification_system.isnull()]
utils.save_df_csv(untyped_inter,name='untyped_interstegmenal_neurons')